In [8]:
# Gathering Stock Market Data

In [4]:
import yfinance as yf

# Download historical stock data
ticker = 'AAPL'  # Example: Apple Inc.
data = yf.download(ticker, start='2020-01-01', end='2024-01-01')
print(data.head())

[*********************100%***********************]  1 of 1 completed

Price                      Adj Close      Close       High        Low  \
Ticker                          AAPL       AAPL       AAPL       AAPL   
Date                                                                    
2020-01-02 00:00:00+00:00  72.796021  75.087502  75.150002  73.797501   
2020-01-03 00:00:00+00:00  72.088295  74.357498  75.144997  74.125000   
2020-01-06 00:00:00+00:00  72.662727  74.949997  74.989998  73.187500   
2020-01-07 00:00:00+00:00  72.320969  74.597504  75.224998  74.370003   
2020-01-08 00:00:00+00:00  73.484375  75.797501  76.110001  74.290001   

Price                           Open     Volume  
Ticker                          AAPL       AAPL  
Date                                             
2020-01-02 00:00:00+00:00  74.059998  135480400  
2020-01-03 00:00:00+00:00  74.287498  146322800  
2020-01-06 00:00:00+00:00  73.447502  118387200  
2020-01-07 00:00:00+00:00  74.959999  108872000  
2020-01-08 00:00:00+00:00  74.290001  132079200  


In [9]:
# Preprocessing the Data

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

ModuleNotFoundError: No module named 'sklearn'

In [7]:
pip install scikit-learn

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.0 MB 6.3 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/11.0 MB 5.0 MB/s eta 0:00:02
   --------- ------------------------------ 2.6/11.0 MB 4.2 MB/s eta 0:00:02
   ----------- ---------------------------- 3.1/11.0 MB 3.9 MB/s eta 0:00:02
   ------------- -------------------------- 3.7/11.0 MB 3.8 MB/s eta 0:00:02
   ---------------- ----------------------- 4.5/11.0 MB 3.6 MB/s eta 0:00:02
   ------------------ --------------------- 5.0/11.0 MB 3.5 MB/s eta 0:00:02
   -------------------- ------------------- 5.8/11.0 MB 3.5 MB/s eta 0:00:02
   ---------------------- ----------------- 6.3/11.0 MB 3.4 MB/s eta 0:00:02
   ------------------------ --------------- 6.8/11.0 MB 3.4 MB/s eta 0:00:02
   --------------------------- ------------ 7.6/11.0 MB 3.3 MB/s eta 0:00:02
   ----------------------------- ---------- 8.1/11.0 MB 3.3 MB/s eta 0:00:01
   ---

In [1]:
import sklearn
print(sklearn.__version__)

1.5.2


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [5]:
# Select relevant features
data['Return'] = data['Close'].pct_change()
data = data.dropna()

In [6]:
# Feature scaling
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[['Close', 'Volume', 'Return']])

In [7]:
# Splitting data
train_size = int(len(scaled_data) * 0.8)
train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]

In [10]:
# Building the Machine Learning Model